In [1]:
import tensorflow as tf
from tensorflow.keras import preprocessing

import numpy as np

In [2]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

targets =[[1], [0], [1], [1], [0], [1]]

In [3]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
input_sequences = np.array(sequences)
labels = np.array(targets)

word_index = tokenizer.word_index

In [4]:
from tensorflow.keras import backend
from tensorflow.keras import layers

In [20]:
batch_size = 2
num_epochs = 10

In [6]:
vocab_size = len(word_index) + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

# Keras Sequential

In [7]:
model = tf.keras.Sequential([
layers.Embedding(vocab_size, emb_size, input_length = 4),
layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
layers.Dense(hidden_dimension, activation='relu'),
layers.Dense(output_dimension, activation='sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 128)            2688      
_________________________________________________________________
lambda (Lambda)              (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/10
3/3 [==============================] - 0s 1ms/step - loss: 4.8441e-04 - accuracy: 1.0000
Epoch 2/10
3/3 [==============================] - 0s 1ms/step - loss: 4.7631e-04 - accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 0s 1ms/step - loss: 4.6764e-04 - accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 0s 3ms/step - loss: 4.6038e-04 - accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 0s 800us/step - loss: 4.5245e-04 - accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 0s 790us/step - loss: 4.4411e-04 - accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 0s 808us/step - loss: 4.3627e-04 - accuracy: 1.0000
Epoch 8/10
3/3 [==============================] - 0s 1ms/step - loss: 4.2967e-04 - accuracy: 1.0000
Epoch 9/10
3/3 [==============================] - 0s 1ms/step - loss: 4.2225e-04 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 0s 1ms/step - loss: 4.1613e-04 - accuracy: 

# Keras Functional API

In [10]:
inputs = layers.Input(shape = (4, ))
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis=1)
hidden_layer = layers.Dense(hidden_dimension, activation='relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation='sigmoid')(hidden_layer)

In [11]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 4, 128)            2688      
_________________________________________________________________
tf_op_layer_Mean (TensorFlow [(None, 128)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/10
3/3 [==============================] - 0s 730us/step - loss: 4.7962e-04 - accuracy: 1.0000
Epoch 2/10
3/3 [==============================] - 0s 773us/step - loss: 4.6909e-04 - accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 0s 760us/step - loss: 4.6004e-04 - accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 0s 749us/step - loss: 4.5040e-04 - accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 0s 1ms/step - loss: 4.4213e-04 - accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 0s 1ms/step - loss: 4.3387e-04 - accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 0s 743us/step - loss: 4.2436e-04 - accuracy: 1.0000
Epoch 8/10
3/3 [==============================] - 0s 904us/step - loss: 4.1597e-04 - accuracy: 1.0000
Epoch 9/10
3/3 [==============================] - 0s 1ms/step - loss: 4.0855e-04 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 0s 1ms/step - loss: 4.0042e-04 - accu

# Keras Custom Model

In [16]:
class CustomModel(tf.keras.Model):

    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name='my_model')
        self.embedding = layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation='relu')
        self.output_layer = layers.Dense(output_dimension, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis=1)
        x = self.dense_layer(x)
        x = self.output_layer(x)
        
        return x

In [18]:
model = CustomModel(vocab_size = vocab_size,
            embed_dimension=emb_size,
            hidden_dimension=hidden_dimension,
            output_dimension=output_dimension)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6930 - accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6761 - accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6611 - accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 0s 848us/step - loss: 0.6444 - accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6263 - accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6058 - accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 0s 1ms/step - loss: 0.5812 - accuracy: 1.0000
Epoch 8/10
3/3 [==============================] - 0s 1ms/step - loss: 0.5534 - accuracy: 1.0000
Epoch 9/10
3/3 [==============================] - 0s 1ms/step - loss: 0.5196 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 0s 1ms/step - loss: 0.4802 - accuracy: 1.0000


# Keras Custom Layer

In [22]:
class CustomLayer(layers.Layer):

    def __init__(self, hidden_dimension, output_dimension, **kwargs):
        self.hidden_dimension = hidden_dimension
        self.output_dimension = output_dimension
        super(CustomLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.dense_layer1 = layers.Dense(self.hidden_dimension, activation = 'relu')
        self.dense_layer2 = layers.Dense(self.output_dimension)

    def call(self, inputs):
        hidden_output = self.dense_layer1(inputs)
        return self.dense_layer2(hidden_output)

    # Optional
    def get_config(self):
        base_config = super(CustomLayer, self).get_config()
        base_config['hidden_dim'] = self.hidden_dimension
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [23]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, emb_size, input_length = 4),
    layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
    CustomLayer(hidden_dimension, output_dimension),
    layers.Activation('sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6956 - accuracy: 0.3333
Epoch 2/10
3/3 [==============================] - 0s 804us/step - loss: 0.6767 - accuracy: 0.8333
Epoch 3/10
3/3 [==============================] - 0s 846us/step - loss: 0.6615 - accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 0s 842us/step - loss: 0.6470 - accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6294 - accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 0s 1ms/step - loss: 0.6090 - accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 0s 960us/step - loss: 0.5850 - accuracy: 1.0000
Epoch 8/10
3/3 [==============================] - 0s 1ms/step - loss: 0.5593 - accuracy: 1.0000
Epoch 9/10
3/3 [==============================] - 0s 1ms/step - loss: 0.5293 - accuracy: 1.0000
Epoch 10/10
3/3 [==============================] - 0s 947us/step - loss: 0.4915 - accuracy: 1.0000
